# Consolidation Notebook

Oct 9, 2021


## Import & Preparation of Notebook

In [3]:
import numpy as np
import pandas as pd
from tqdm import trange

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions

import probflow as pf

# Data Viz. 
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

dtype = 'float32'

# Get TensorFlow version.
print(f'TnesorFlow version: {tf.__version__}')
print(f'TnesorFlow Probability version: {tfp.__version__}')

if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

TnesorFlow version: 2.4.1
TnesorFlow Probability version: 0.12.1


2021-10-13 07:18:14.941468: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data

In [4]:
xname="sm_0.67"; yname="halo_mass"
xname="sm_1.0"; yname="halo_mass"; x2name="central_sm"

gal_df = pd.read_csv("Data/galaxies_near_clusters_0.3-0.6.csv") 
cluster_data = pd.read_csv("Data/cluster_data_0.3-0.6.csv")
x=cluster_data[xname]; 
y=cluster_data[yname];
x2=cluster_data[x2name]


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
# Set seed.
tf.random.set_seed(42)
# Set tensor numeric type.
dtype = 'float32'

scaler = StandardScaler()
detector = IsolationForest(n_estimators=1000,  contamination="auto", random_state=0)

# split into train and test sets
X=np.vstack([x,x2]).transpose()
X_train, X_test,  y_train, y_test = train_test_split(X, y, train_size=0.67)
print("train:  ",X_train.shape, y_train.shape," test:  ",  X_test.shape, y_test.shape)
#print(X_train[:,0].shape, X_train[:,1].shape,y_train.shape)
 
# Scaling
unit_df =pd.DataFrame(data={xname:X_train[:,0], x2name:X_train[:,1], yname:y_train})
# Scale data to zero mean and unit variance.
X_t = scaler.fit_transform(unit_df)
print("X_t shape",X_t.shape)

# Remove outliers.
detector = IsolationForest(n_estimators=1000,  contamination=0.15, random_state=0)
is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier > 0),:]
unit_df =pd.DataFrame(data={xname:X_t[:,0], x2name:X_t[:,1], yname:X_t[:,2]})

# Inverse scaling
X_t = scaler.inverse_transform(unit_df)
inv_df=pd.DataFrame(data={xname:X_t[:,0], x2name:X_t[:,1], yname:X_t[:,2]})
xc=X_t[:,0]
x2c=X_t[:1]
yc=X_t[:,2]

# Tensors for tensorflow
# all the data
x = np.vstack([cluster_data[xname],cluster_data[x2name]]).transpose()
x = tf.convert_to_tensor(x, dtype=dtype)

y = tf.convert_to_tensor(cluster_data[yname], dtype=dtype)
y = tf.reshape(y, (-1, 1))

# Just the train data
x = np.vstack([inv_df[xname],inv_df[x2name]]).transpose()
x = tf.convert_to_tensor(x, dtype=dtype)

y = tf.convert_to_tensor(inv_df[yname], dtype=dtype)
y = tf.reshape(y, (-1, 1))

print("x shape: {}, y shape: {}".format(x.shape,y.shape))

train:   (192, 2) (192,)  test:   (95, 2) (95,)
X_t shape (192, 3)


2021-10-13 07:18:29.160877: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-13 07:18:29.163440: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 32. Tune using inter_op_parallelism_threads for best performance.


x shape: (163, 2), y shape: (163, 1)


## Model libraries

### Linear fit
A linear regression can be expressed as model with only a single perceptron tf.keras.layers.Dense(1), which by default has a linear activation function and can take bias into account (that is, for x=0 the model will not necessarily predict y=0).

- model = tf.keras.Sequential([    tf.keras.layers.Dense(1)  ])

### Gaussians for each point

Let’s try now to make the model output not just a number, but a distribution; for the sake of definiteness, let’s try to learn the “best” normal distribution which models the random variable Y for each value of X . In order to do so, notice that we cannot use anymore the mean squared error as a loss function, since it is not a meaningful error metric in this case. We will, instead, use the negative logarithm of likelihood between the distributions given by our model vs the observed data; in this way, training our model becomes essentially a maximum likelihood estimation of parameters!

Notice also that for each value of X the normal distribution we want our model to learn is parametrized by two numbers: mean and standard deviation. Therefore, we should increase the number of neurons in the dense layer from one to two, so that each perceptron will learn the functional dependency on X of one of the two parameters, and then push its output into the distribution.

The way to implement these changes in TensorFlow Probability is very nice: we can use a tfp.layers.DistributionLambda layer which works in pretty much the same way as a “standard” Keras layer; in its argument, we can plug a lambda function which takes parameters from the previous layers of the network and returns a tfp.Distribution:

- model = tf.keras.Sequential([
            tf.keras.layers.Dense(2),
            tfp.layers.DistributionLambda(lambda t:
                tfp.distributions.LogNormal( loc   = t[...,:1], scale = tf.math.softplus(0.005*t[...,1:])+0.001))   ])
                
- negloglik = lambda y, p_y: -p_y.log_prob(y)

### Case 1: No Uncertainty

Seems like the above. The argument to Dense() is units: Positive integer, dimensionality of the output space.

-  model = tf.keras.Sequential([
          tf.keras.layers.Dense(1),
          tfp.layers.DistributionLambda(lambda t: 
              tfd.Normal(loc=t, scale=1)), ])

### Case 2: Aleatoric Uncertainty

- model = tf.keras.Sequential([
          tf.keras.layers.Dense(1 + 1),
          tfp.layers.DistributionLambda(lambda t: 
              tfd.Normal(loc=t[..., :1], scale=1e-3 + tf.math.softplus(0.05 * t[...,1:]))),])
              
### Case 3: Epistemic Uncertainty

- model = tf.keras.Sequential([
          tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight=1/x.shape[0]),
          tfp.layers.DistributionLambda(lambda t: 
              tfd.Normal(loc=t, scale=1)),])
plus models for the prior and posterior

- def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
      n = kernel_size + bias_size
      c = np.log(np.expm1(1.))
      return tf.keras.Sequential([
          tfp.layers.VariableLayer(2 * n, dtype=dtype),
          tfp.layers.DistributionLambda(lambda t: tfd.Independent(
              tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])), reinterpreted_batch_ndims=1)),])
  
- def prior_trainable(kernel_size, bias_size=0, dtype=None):
      n = kernel_size + bias_size
      return tf.keras.Sequential([
          tfp.layers.VariableLayer(n, dtype=dtype),
          tfp.layers.DistributionLambda(lambda t: tfd.Independent(
              tfd.Normal(loc=t, scale=1), reinterpreted_batch_ndims=1)),])
              
### Case 4: Aleatoric & Epistemic Uncertainty

- model = tf.keras.Sequential([
          tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_trainable, kl_weight=1/x.shape[0]),
          tfp.layers.DistributionLambda(lambda t: 
              tfd.Normal(loc=t[..., :1],
                  scale=1e-3 + tf.math.softplus(0.01 * t[...,1:]))),])
                  

### Aleatoric Uncertainty (bnn2-for-stellarmass)

To account for aleotoric uncertainty, which arises from the noise in the output, dense layers are combined with probabilistic layers. More specifically, the mean and covariance matrix of the output is modelled as a function of the input and parameter weights. 

The first hidden layer shall consist of ten nodes, the second one needs four nodes for the means plus ten nodes for the variances and covariances of the four-dimensional (there are four outputs) multivariate Gaussian posterior probability distribution in the final layer. This is achieved using the params_size method of the last layer (MultivariateNormalTriL), which is the declaration of the posterior probability distribution structure, in this case a multivariate normal distribution in which only one half of the covariance matrix is estimated (due to symmetry). The total number of parameters in the model is 224 — estimated by variational methods. The deterministic version of this neural network consists of an input layer, ten latent variables (hidden nodes), and an output layer (114 parameters), which does not include the uncertainty in the parameters weights.

- model = tfk.Sequential([
            tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"),
            # just one output (mean, var), so 10->2
            tfk.layers.Dense(2, activation="relu", name="dense_1"),
            tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(
                len(outputs)), activation=None, name="distribution_weights"),
            tfp.layers.MultivariateNormalTriL(len(outputs), 
                activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches), name="output")],)

#### Aleotoric and Epistemic Uncertainty

To account for aleotoric and epistemic uncertainty (uncertainty in parameter weights), the dense layers have to be exchanged with Flipout layers (DenseFlipout) or with Variational layers (DenseVariational). Such a model has more parameters, since every weight is parametrized by normal distribution with non-shared mean and standard deviation, hence doubling the amount of parameter weights. 
Weights will be resampled for different predictions, and in that case, the Bayesian neural network will act like an ensemble.
                
- tfp.layers.DenseFlipout(10, activation="relu", name="dense_1")

#### Adding uncertainity for the geosciences (bnn3-for-stellarmass)

- output_layer = tf.keras.layers.concatenate(
        [mu_unit , logsigma_unit , skew_unit , logtau_unit], axis=1)

- model = tf.keras.models.Model(inputs=inputs , outputs=output_layer)

initialize a single hidden layer
- x = tf.keras.layers.Dense( N_HIDDENS,
        activation="relu",
        use_bias=True,
        bias_initializer=RandomNormal(seed=SEED),
        kernel_initializer=RandomNormal(seed=SEED), )(x)
 
set final output units separately
-  mu_unit = tf.keras.layers.Dense( 1,
        activation="linear",
        use_bias=True, 
        bias_initializer=RandomNormal(seed=SEED), 
        kernel_initializer=RandomNormal(seed=SEED),)(x)
 
 - logsigma_unit = tf.keras.layers.Dense( 1,
        activation="linear", 
        use_bias=True, 
        bias_initializer=Zeros(), 
        kernel_initializer=Zeros(),)(x)
 
 
 #### Prediction Intervals for Deep Learning Neural Networks (bnn6-for-stellarmass)
 
Next, we can define, train and evaluate a Multilayer Perceptron (MLP) model on the dataset. We will define a simple model with two hidden layers and an output layer that predicts a numeric value. We will use the ReLU activation function and “he” weight initialization, which are a good practice. The number of nodes in each hidden layer was chosen after a little trial and error.
 
- model = tf.keras.Sequential()
- model.add(tf.keras.layers.Dense(20, kernel_initializer='he_normal', activation='relu', input_dim=features))
- model.add(tf.keras.layers.Dense(5, kernel_initializer='he_normal', activation='relu'))
- model.add(tf.keras.layers.Dense(1))
 
 #### Simple Bayesian Linear Regression with TensorFlow Probability (bb7-for-stellarmass)
 
 - jds_ab = tfd.JointDistributionNamedAutoBatched(
        dict(
            sigma=tfd.HalfNormal(scale=[tf.cast(1.0, dtype)]),
            alpha=tfd.Normal( loc=[tf.cast(0.0, dtype)],  scale=[tf.cast(10.0, dtype)]),
            beta=tfd.Normal( 
                loc=[[tf.cast(0.0, dtype)], [tf.cast(0.0, dtype)]], 
                scale=[[tf.cast(10.0, dtype)], [tf.cast(10.0, dtype)]]),
            y=lambda beta, alpha, sigma: 
                tfd.Normal( loc=tf.linalg.matmul(x, beta) + alpha, scale=sigma) ))
               
 - uses a Hamiltonian monte carlo to explore the probability distributions
 
 It's unclear how this HMC approach relates to the epistemic model probabilities I looked at earlier.
 
 But perhaps the fragment presented next is how to think about it.
 
 ### Bayesian Regressions with MCMC or Variational Bayes using TensorFlow Probability
 
 Brendan Hasz, 3 Dec 2018, https://brendanhasz.github.io/2018/12/03/tfp-regression.html
 
 Variational Bayes

Despite its ability to accurately capture uncertainty, one big problem with MCMC sampling is that it’s slow. It took around 20s to sample from the posterior of our simple model with only 100 datapoints! That might not sound too bad, but speed becomes huge when trying to fit models to real (i.e. large) datasets, especially when the model is more complex.

To speed things up while still capturing uncertainty, we can make some simplifying assumptions and use stochastic variational inference. These assumptions are that our model’s parameters are completely independent of each other (which MCMC sampling does not assume, since it samples from the joint distribution), and that the posterior for each of our parameters follow some simple distribution (like a normal distribution).

How do these assumptions change how we can infer the model parameters? Instead of sampling from the posterior a bunch of times like we did with MCMC (which takes too long), and instead of simply optimizing the parameters (which only gets us a single best point estimate and no uncertainty information), we’ll compromise. **We’ll use stochastic gradient descent to optimize the parameters, but instead of just optimizing the parameter values, we’ll replace each single parameter value with a simple distribution. We’ll replace our point estimates of the weight and bias parameters with normal distributions.** This doubles the numbers of parameters in the model, because now we have a mean parameter and a variance parameter for each single parameter we had before. At each training step, we’ll sample a value from that normal distribution to get the value of the parameter for that training step. Then we can update all the mean and variance parameters (for each original parameter) through backpropagation, instead of updating the point value of the parameter. We’ll also have to use the evidence lower bound (ELBO) loss function, which includes both the loss due to the expected log probability and the difference between the parameter’s distributions and their priors. I’ll do a post on that sometime in the future, but for now see this page.

**The advantage of stochastic variational inference over simply optimizing the parameters’ values is that we capture uncertainty as to the model parameter values (because we’re treating each parameter as a distribution now, which has some variance)**. An added bonus is that stochastically sampling from the variational distributions regularizes our model, helping to prevent overfitting. However, while it’s faster than MCMC sampling, it also makes more assumptions than MCMC sampling does (about the shape of the individual parameters’ posteriors and their independence).

We’ll use a super-simple one-layer Bayesian neural network with no activation (i.e., a linear regression, which is equivalent to the model we fit with MCMC earlier) and see if it captures the same level of uncertainty as the previous model which we fit with MCMC.


## What do we want?

The same model that Orduz uses, but with many sigma

- jds_ia = tfd.JointDistributionSequential([
        tfd.Normal(loc=0., scale=1.),   # m
        tfd.Normal(loc=0., scale=1.),   # b
        lambda b, m: tfd.Independent(   # Y
            tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], 
            scale=1e-3 + tf.math.softplus(0.01 * t[...,1:]))) ,
            reinterpreted_batch_ndims=1)
])

This looks like 2 guassians for m,n. And then, instead of a single gaussian describing y, it's a series of gaussians.

Looks like I just add the prior and posteriors to do the epstemic.

but first try this model

In [31]:
# A linear model. From Geosciences

jds_ab = tfd.JointDistributionNamedAutoBatched(dict(

    sigma = tfd.HalfNormal(scale=[tf.cast(1.0, dtype)]),

    alpha=tfd.Normal(
        loc=[tf.cast(3.0, dtype)], 
        scale=[tf.cast(4.0, dtype)]
    ),

    beta=tfd.Normal(
        loc=[[tf.cast(0.0, dtype)], [tf.cast(0.0, dtype)]], 
        scale=[[tf.cast(1.0, dtype)], [tf.cast(1.0, dtype)]]
    ),

    y=lambda beta, alpha, sigma: 
        tfd.Normal(
            loc=tf.linalg.matmul(x, beta) + alpha, 
            scale=sigma
        ) 
))

jds_ab.sample(1)["y"]
jds_ab.sample
jds_ab.resolve_graph()
jds_ab.sample()["y"]
print(type(jds_ab))
jds_ab.event_shape_tensor()
pjds_ab=jds_ab.experimental_pin(y=y)

<class 'tensorflow_probability.python.distributions.joint_distribution_auto_batched.JointDistributionNamedAutoBatched'>


In [25]:
@tfd.JointDistributionCoroutineAutoBatched
def model():
 uranium_weight = yield tfd.Normal(0., scale=1., name='uranium_weight')
 county_floor_weight = yield tfd.Normal(
     0., scale=1., name='county_floor_weight')
 county_effect = yield tfd.Sample(
     tfd.Normal(0., scale=county_effect_scale),
     sample_shape=[num_counties], name='county_effect')
 yield tfd.Normal(
     loc=(log_uranium * uranium_weight
          + floor_of_house * floor_weight
          + floor_by_county * county_floor_weight
          + tf.gather(county_effect, county, axis=-1)
          + bias),
     scale=log_radon_scale[..., tf.newaxis],
     name='log_radon')
print(type(model))

<class 'tensorflow_probability.python.distributions.joint_distribution_auto_batched.JointDistributionCoroutineAutoBatched'>


In [32]:
# A linear model. From Geosciences

jds_ab = tfd.JointDistributionNamedAutoBatched(dict(

    sigma = tfd.HalfNormal(scale=[tf.cast(1.0, dtype)]),

    alpha=tfd.Normal(
        loc=[tf.cast(3.0, dtype)], 
        scale=[tf.cast(4.0, dtype)]
    ),

    beta=tfd.Normal(
        loc=[[tf.cast(0.0, dtype)], [tf.cast(0.0, dtype)]], 
        scale=[[tf.cast(1.0, dtype)], [tf.cast(1.0, dtype)]]
    ),

    y=lambda beta, alpha, sigma: 
        tfd.Normal(
            loc=tf.linalg.matmul(x, beta) + alpha, 
            scale=sigma
        ) 
))
hidden_all = tf.keras.layers.Dense(N_Hidden, activation="relu",
            use_bias=True, 
            bias_initializer=tf.keras.initializers.RandomNormal(), 
            kernel_initializer=tf.keras.initializers.RandomNormal(),
            name="hidden",
         )


N_Hidden=10
inputs = tf.keras.Input(x.shape[0], name="input") 
# initialize a single hidden layer
hidden = tf.keras.layers.Dense(N_Hidden, activation="relu", use_bias=True, name="hidden",) (inputs)

output_layer = tfp.layers.DistributionLambda(
    make_distribution_fn=lambda t:pjds_ab(), 
    convert_to_tensor_fn=lambda s:pjds_ab.sample()["y"])
print(type(output_layer))

model = tf.keras.models.Model(inputs=inputs, outputs=output_layer)

<class 'tensorflow_probability.python.layers.distribution_layer.DistributionLambda'>


ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <tensorflow_probability.python.layers.distribution_layer.DistributionLambda object at 0x2aac0f40c0d0>

In [32]:
tf.python.keras.layers.Lambda(output_layer)


AttributeError: module 'tensorflow.compat.v2' has no attribute 'python'

In [7]:
jds_ab.resolve_graph()

(('sigma', ()), ('beta', ()), ('alpha', ()), ('y', ('beta', 'alpha', 'sigma')))

In [17]:
jds_ab.sample()
#jds_ab.parameter_properties()
#jds_ab.experimental_fit(xc)
tf.keras.layers.Layer(jds_ab)

In [14]:
model = tf.keras.models.Sequential(jds_ab)

TypeError: The added layer must be an instance of class Layer. Found: tfp.distributions.JointDistributionNamedAutoBatched("JointDistributionNamedAutoBatched", batch_shape=[], event_shape={alpha: [1], beta: [2, 1], sigma: [1], y: [163, 1]}, dtype={alpha: float32, beta: float32, sigma: float32, y: float32})

In [19]:
def target_log_prob_fn(beta, alpha, sigma):
    return jds_ab.log_prob(beta=beta, alpha=alpha, sigma=sigma, y=y)

# Do inference.
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=target_log_prob_fn)
model.fit(x, y, epochs=1000, verbose=False);

# Profit.
[print(np.squeeze(w.numpy())) for w in model.weights];
yhat = model(x_tst)
assert isinstance(yhat, tfd.Distribution)

AttributeError: 'JointDistributionNamedAutoBatched' object has no attribute 'compile'